<a href="https://colab.research.google.com/github/ProfAI/dl00/blob/master/9%20-%20Reti%20neurali%20ricorrenti%20(non%20rilasciata)/LSTM_e_GRU_con%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM e GRU con Keras

Le reti **Long short-term memory (LSTM)** sono un'architettura di reti neurali ricorrenti che risolvono il problema della scomparsa del gradiente tra le diverse esecuzioni della rete.
<br>
Le LSTM  sono state da Sepp Hochreiter e Jurger Schmidhuber nel 1997 in [questa ricerca](https://www.bioinf.jku.at/publications/older/2604.pdf) ed in sostanza aggiungono canale prioritario alla rete, chiamato cell state o memory cell, che viaggia in parallelo al segnale della rete e immagazzina le informazioni sequenziali.
<br><br>
In questo notebook utilizzeremo le LSTM per migliorare la rete neurale per la sentiment analysis addestrata sull'IMDB Movie Reviews dataset.

In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset
Utilizziamo Keras per caricare l'imdb dataset, limitandolo alle 10000 parole più comuni.

In [5]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati
Le recensioni all'interno del corpus di testo hanno ovviamente lunghezza differente, utilizziamo la funzione pad_sequences di keras per limitare le sequenze a 500 elementi (nel nostro caso limitare le frasi a 500 parole).
Se una sequenza ha meno di 500 esempi verranno aggiunti degli zeri alla fine.


In [6]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [3]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 500)

## Creiamo il modello LSTM
Costruiamo il nostro modello strutturandolo così:
1. Il primo strato eseguirà l'embedding creando 50 embedding vectors per ognuna delle 10.000 parole nel nostro dizionario.
2. Il secondo strato è lo strato ricorrente di tipo Long-short term memory.
3. Il terzo strato calcolerà l'ouput della rete, trattandosi di un problema di classifcazione binaria (recensione positiva/negativa) la funzione di attivazione sarà la sigmoide.

In [5]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 5 epoche.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 37s 2ms/step - loss: 0.6412 - acc: 0.6766 - val_loss: 0.5538 - val_acc: 0.7566
Epoch 2/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.4363 - acc: 0.8274 - val_loss: 0.3925 - val_acc: 0.8382
Epoch 3/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.3244 - acc: 0.8763 - val_loss: 0.4741 - val_acc: 0.8024
Epoch 4/5
20000/20000 [==============================] - 34s 2ms/step - loss: 0.2708 - acc: 0.8995 - val_loss: 0.3731 - val_acc: 0.8464
Epoch 5/5
25000/25000 [==============================] - 119s 5ms/step


[0.32797487154483795, 0.87216]

Il risultato è nettamente migliore rispetto ad una semplice RNN, c'è un po' di overfitting, proviamo a correggerlo utilizzando il dropout.

## Dropout in una RNN
In una RNN possiamo utilizzare due differenti tipologie di dropout.
1. Dropout sull'input/output dello strato, esattamente come abbiamo già fatto con le altre architetture di reti neurali.
2. Dropout tra le esecuzioni ricorrenti della rete, questo permette di ridurre l'overfitting nelle features che contengono le informazioni sulla sequenza.

Per utilizzare il dropout sull'input di uno strato ricorrente, piuttosto che usare la classe Dropout, è consigliato sfruttare il parametro dropout delle classi SimpleRNN e LSTM.
Per utilizzare il dropout ricorrente possiamo invece utilizzare il parametro recurrent_dropout delle classi SimpleRNN e LSTM.

In [ ]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 5 epoche.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 41s 2ms/step - loss: 0.6590 - acc: 0.6369 - val_loss: 0.5685 - val_acc: 0.7596
Epoch 2/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.4999 - acc: 0.8003 - val_loss: 0.4337 - val_acc: 0.8184
Epoch 3/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.4029 - acc: 0.8409 - val_loss: 0.3740 - val_acc: 0.8418
Epoch 4/5
20000/20000 [==============================] - 40s 2ms/step - loss: 0.3614 - acc: 0.8567 - val_loss: 0.3482 - val_acc: 0.8534
Epoch 5/5
25000/25000 [==============================] - 139s 6ms/step


[0.4099230393791199, 0.81852]

Abbiamo ridotto l'overfitting nella nostra rete, adesso proviamo a migliorare il modello aggiungendo degli altri strati ricorrenti alla rete.

## Aggiungiamo altri strati LSTM
Possiamo aggiungere altri strati ricorrenti ad una rete nella solita maniera, utilizzando il metodo add della classe Sequential.
Tieni conto solo di una cosa, di default la classe LSTM esegue il flattening della sequenza per poterla dare come input ad uno strato denso, possiamo modificare tale comportamento impostando il parametro return_sequences a True.

In [10]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 32)          17024     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,025,377
Trainable params: 1,025,377
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 5 epoche, questa volta cronometriamo la durata totale dell'addestramento, ci servirà dopo per eseguire il confronto con la variante GRU.

In [11]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
start_at = time()
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
print("Addestramento completato in %.f secondi (5 epoche)" % ((time()-start_at)))
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 79s 4ms/step - loss: 0.6242 - acc: 0.6494 - val_loss: 0.6510 - val_acc: 0.7038
Epoch 2/5
20000/20000 [==============================] - 77s 4ms/step - loss: 0.4499 - acc: 0.8103 - val_loss: 0.3923 - val_acc: 0.8314
Epoch 3/5
20000/20000 [==============================] - 76s 4ms/step - loss: 0.3799 - acc: 0.8471 - val_loss: 0.4720 - val_acc: 0.7824
Epoch 4/5
20000/20000 [==============================] - 78s 4ms/step - loss: 0.3406 - acc: 0.8705 - val_loss: 0.3780 - val_acc: 0.8366
Epoch 5/5
20000/20000 [==============================] - 77s 4ms/step - loss: 0.3191 - acc: 0.8780 - val_loss: 0.3822 - val_acc: 0.8388
Addestramento completato in 390 secondi (5 epoche)
25000/25000 [==============================] - 275s 11ms/step


[0.3829597888946533, 0.83548]

## Gated Recurrent Unit (GRU)

Le **Gated Recurrent Unit (GRU)** sono una tipologia di reti neurali ricorrenti che prendono spunto da e semplificano le LSTM.
<br>
A differenza di quest'ultime le GRU richiedono meno calcoli tensoriali e quindi solitamente portano a risultati simili in minor tempo.
<br>
Possiamo aggiungere degli strati GRU con Keras tramite la classe GRU, facciamolo utilizzando la stessa architettura della rete LSTM profonda.

In [12]:
from keras.layers import Embedding, GRU, Dropout

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(GRU(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(GRU(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 32)          12768     
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,019,041
Trainable params: 1,019,041
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello e avviamo l'addestramento per 5 epoche, cronometrandone la durata.

In [13]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
start_at = time()
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
print("Addestramento completato in %.f secondi (5 epoche)" % ((time()-start_at)))
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 65s 3ms/step - loss: 0.6584 - acc: 0.6056 - val_loss: 0.5248 - val_acc: 0.7350
Epoch 2/5
20000/20000 [==============================] - 61s 3ms/step - loss: 0.4874 - acc: 0.7753 - val_loss: 0.4472 - val_acc: 0.7884
Epoch 3/5
20000/20000 [==============================] - 61s 3ms/step - loss: 0.4101 - acc: 0.8266 - val_loss: 0.4207 - val_acc: 0.8116
Epoch 4/5
20000/20000 [==============================] - 61s 3ms/step - loss: 0.3700 - acc: 0.8498 - val_loss: 0.4369 - val_acc: 0.8012
Epoch 5/5
20000/20000 [==============================] - 63s 3ms/step - loss: 0.3383 - acc: 0.8646 - val_loss: 0.4283 - val_acc: 0.8092
Addestramento completato in 313 secondi (5 epoche)
25000/25000 [==============================] - 211s 8ms/step


[0.4191904431438446, 0.8156]

Il risultato è livemente inferiore a quello ottenuto con le LSTM ma l'addestramento ha richiesto il 20% del tempo in meno.